## Category-Level Pricing Feasibility And Strategy
### Objective
The objective of this notebook is to translate pricing and inventory insights into category-specific pricing strategies.

Rather than applying a single pricing decision across the entire business, this analysis evaluates how different product categories respond to pricing changes under inventory constraints.

This notebook answers:

- Which categories can safely support discounts?

- Which categories face high stockout risk?

- Where should pricing be aggressive vs conservative?

### Business Context

Retail pricing decisions are rarely uniform. Categories differ in:

- Demand elasticity

- Inventory availability

- Revenue sensitivity

A pricing strategy that works for Electronics may be disastrous for Grocery.

This notebook converts analytics into category-level executive decisions.

### Import Libraries

In [ ]:
import pandas as pd #importing pandas for data manipulation
import numpy as np #importing numpy for mathematical calculations

###Load Featured Dataset

This dataset already includes:

- Pricing scenarios

- Predicted demand

- Feasible (inventory-adjusted) revenue

- Stockout indicators

In [ ]:
df= pd.read_csv("ecommerce_pricing_featured_02.csv")
df.head()

,product_id,category,base_price,competitor_price,promotion_flag,season,customer_segment,day_of_week,channel,current_price,...,fulfilled_minus_10,fulfilled_plus_5,fulfilled_plus_10,feasible_revenue_baseline,feasible_revenue_minus_10,feasible_revenue_plus_5,feasible_revenue_plus_10,stock_minus_10,stock_plus_5,stock_plus_10
0,1102,Electronics,153.93,23.94,True,Fall,Regular,Tue,Web,111.99,...,602.000000,602.000000,602.000000,67417.980000,60676.182000,70788.879000,74159.778000,True,True,True
1,1435,Home,190.17,250.84,False,Spring,Premium,Sat,Mobile App,190.17,...,327.056638,137.952858,74.918265,38221.783673,55976.724816,27546.219808,15671.927082,False,False,False
2,1860,Electronics,166.57,257.71,False,Summer,Regular,Mon,Web,166.57,...,467.862734,302.226637,247.014604,59538.559139,70138.706071,52858.985435,45259.744907,False,False,False
3,1270,Electronics,215.97,238.38,False,Winter,Regular,Mon,Web,215.97,...,173.124550,-41.634578,-113.220954,6468.689904,33650.738241,-9441.410709,-26897.562284,False,False,False
4,1106,Fashion,102.55,255.81,True,Spring,Regular,Mon,Web,81.71,...,571.000000,571.000000,571.000000,46656.410000,41990.769000,48989.230500,51322.051000,True,True,True


#### Interpretation

We reuse the engineered dataset to ensure consistency across all pricing analyses. This reflects how analytics teams build layered insights rather than restarting analysis from scratch.

### Step 1 — Aggregate Revenue by Category & Scenario

In [ ]:
category_summary= df.groupby('category').agg({'feasible_revenue_baseline': 'sum',
                                              'feasible_revenue_minus_10': 'sum',
                                              'feasible_revenue_plus_5': 'sum',
                                              'feasible_revenue_plus_10': 'sum'}).reset_index()
category_summary

,category,feasible_revenue_baseline,feasible_revenue_minus_10,feasible_revenue_plus_5,feasible_revenue_plus_10
0,Electronics,5.805157e+07,1.001056e+08,3.274340e+07,4.575926e+06
1,Fashion,8.594087e+07,1.356119e+08,5.588191e+07,2.233593e+07
2,Grocery,5.906253e+07,8.929886e+07,4.065460e+07,2.008023e+07
3,Home,7.029549e+07,1.105358e+08,4.590656e+07,1.869709e+07


#### Interpretation

This table shows real, achievable revenue per category under each pricing strategy.

Key idea:

- These are not theoretical numbers

- Inventory constraints are already applied

- Executives care about what can actually be earned, not what looks good on paper.

Across all categories, the -10% price cut generates the highest feasible revenue

### Step 2 — Identify Best Pricing Scenario per Category

In [ ]:
scenario_cols=['feasible_revenue_baseline', 'feasible_revenue_minus_10', 'feasible_revenue_plus_5', 'feasible_revenue_plus_10']
category_summary['Best_Scenario']= category_summary[scenario_cols].idxmax(axis=1)
category_summary

,category,feasible_revenue_baseline,feasible_revenue_minus_10,feasible_revenue_plus_5,feasible_revenue_plus_10,Best_Scenario
0,Electronics,5.805157e+07,1.001056e+08,3.274340e+07,4.575926e+06,feasible_revenue_minus_10
1,Fashion,8.594087e+07,1.356119e+08,5.588191e+07,2.233593e+07,feasible_revenue_minus_10
2,Grocery,5.906253e+07,8.929886e+07,4.065460e+07,2.008023e+07,feasible_revenue_minus_10
3,Home,7.029549e+07,1.105358e+08,4.590656e+07,1.869709e+07,feasible_revenue_minus_10


All categories are price-sensitive, where controlled discounts unlock higher realized revenue — provided inventory constraints are respected.

### Step 3 — Compute Category-Level Stockout Rates

In [ ]:
stockout_summary= df.groupby('category').agg({'stock_minus_10': 'mean', 'stock_plus_5': 'mean', 'stock_plus_10': 'mean'}).reset_index()
stockout_summary

,category,stock_minus_10,stock_plus_5,stock_plus_10
0,Electronics,0.339728,0.287060,0.271015
1,Fashion,0.328217,0.274351,0.258722
2,Grocery,0.354300,0.296106,0.281985
3,Home,0.343981,0.287492,0.273033


What this means
mean() of a boolean column = percentage of rows with stockouts

Example:

0.34 = 34% of demand exceeds inventory

Observed stockout rates
- For -10%, the observed stockout rates are 33–35%
- For +5%, the observed stockout rates are 27–29%
- For +10%	the observed stockout rates are 25–28%

Discounts increase demand faster than inventory can respond

### Step 4 — Assign Risk Labels by Category

In [ ]:
def risk_label(rate):
  if rate < 0.10:
    return "Low"
  elif rate < 0.30:
    return "Medium"
  else:
    return "High"
for col in ['stock_minus_10', 'stock_plus_5', 'stock_plus_10']:
  stockout_summary[col+ '_risk']= stockout_summary[col].apply(risk_label)
stockout_summary

,category,stock_minus_10,stock_plus_5,stock_plus_10,stock_minus_10_risk,stock_plus_5_risk,stock_plus_10_risk
0,Electronics,0.339728,0.287060,0.271015,High,Medium,Medium
1,Fashion,0.328217,0.274351,0.258722,High,Medium,Medium
2,Grocery,0.354300,0.296106,0.281985,High,Medium,Medium
3,Home,0.343981,0.287492,0.273033,High,Medium,Medium


Stockout Rate	Business Interpretation
- (<10%)	Operationally safe
- (10–30%)	Manageable but risky
- (>30%)	Severe revenue leakage

The results:

10% price → HIGH risk

+5%, +10% → MEDIUM risk

### Step 5 — Merge Revenue & Risk Views

In [ ]:
category_decision_table= category_summary.merge(stockout_summary, on='category', how='left')
category_decision_table

,category,feasible_revenue_baseline,feasible_revenue_minus_10,feasible_revenue_plus_5,feasible_revenue_plus_10,Best_Scenario,stock_minus_10,stock_plus_5,stock_plus_10,stock_minus_10_risk,stock_plus_5_risk,stock_plus_10_risk
0,Electronics,5.805157e+07,1.001056e+08,3.274340e+07,4.575926e+06,feasible_revenue_minus_10,0.339728,0.287060,0.271015,High,Medium,Medium
1,Fashion,8.594087e+07,1.356119e+08,5.588191e+07,2.233593e+07,feasible_revenue_minus_10,0.328217,0.274351,0.258722,High,Medium,Medium
2,Grocery,5.906253e+07,8.929886e+07,4.065460e+07,2.008023e+07,feasible_revenue_minus_10,0.354300,0.296106,0.281985,High,Medium,Medium
3,Home,7.029549e+07,1.105358e+08,4.590656e+07,1.869709e+07,feasible_revenue_minus_10,0.343981,0.287492,0.273033,High,Medium,Medium


In [ ]:
category_decision_table.to_csv("category_decision_table.csv", index=False)

### Final Interpretation
#####Category-Level Findings

- All categories exhibit high price elasticity

- A -10% price reduction maximizes feasible revenue across categories

- However, this strategy introduces high stockout risk (33–35%)

#####Strategic Tradeoff

- Revenue upside vs operational feasibility

- Pure discounting is not sustainable without inventory coordination

###Final Business Recommendations
#####Pricing Strategy

- Avoid blanket -10% discounts without inventory buildup

- Use targeted discounting on high-inventory SKUs

- Prefer +5% pricing for categories with replenishment constraints

##### Inventory Strategy

- Increase safety stock ahead of promotions

- Align replenishment cycles with pricing campaigns

- Use demand forecasts to pre-position inventory

Executive Takeaway

- The most profitable pricing strategy on paper is not always the most executable.
- Sustainable revenue growth requires pricing decisions that respect inventory realities.